EJERCICIO 2   

Apoyándose en la sección de información. Separa en fragmentos un texto extenso extraído 
y vectoriza cada fragmento con alguno de los modelos de embedding vistos en clases.  
 
Luego  realiza  un  análisis  de  similitud  de  texto  ingresando  varias  frases  a  buscar 
semánticamente, compare distintas técnicas de distancias vistas en clases, elija la mejor y 
justifique la razón por la que esa técnica se ajusta para este tipo de búsquedas. 
 
OPCIONAL: Visualizar en 3D aplicando PCA o t-SNE la ubicación de los fragmentos y la 
query ingresada vectorizada en el espacio. Realizar una observación sobre la visualización.

Para este ejercicio vamos a usar texto de una reseña sobre el juego de la pagina misutmeeple.com  
Va a ser necesario hacerle un procesamiento previo al fragmentado ya que el formato del texto extraido es inconsistente y contiene información no útil.

In [1]:
#Guardar el contenido del archivo en una variable
with open("./datos/informacion/misutmeeple.com_2018_08_resena-sagrada.txt", 'r') as f:
    reseña = f.read()

In [2]:
# Importar la librería re para expresiones regulares
import re

In [3]:
#Primero, vamos a quedarnos con el contenido de los parrafos
inicio = reseña.lower().find("párrafos:")
aux = reseña[inicio + len("párrafos:") + 1:]
final = aux.lower().find("92.")
parrafos = aux[:final]
#Vamos a eliminar la enumeracion de los parrafos
texto_parrafos = re.sub(r'^\d+\.\s*', '', parrafos, flags=re.MULTILINE)


In [5]:
#Segundo, vamos a quedarnos con el contenido de los comentarios

comentarios = re.findall( #Encuentra todas las apariciones de la expresión regular
    r"COMENTARIO:\s*(.+?)(?=\n\d+\. AUTOR:|\nElementos de listas)",  #Matchea todo desde "COMENTARIO:" hasta el siguiente "AUTOR:" o "Elementos de listas"
    reseña, 
    re.DOTALL # Permite que el punto coincida con saltos de línea, ya que los comentarios pueden abarcar varias líneas
)
# Unir todos los comentarios con saltos de línea
texto_comentarios = "\n".join(comentario.strip() for comentario in comentarios)

In [6]:
#Por ultimo, vamos a quedarnos con el contenido de los elementos de listas

pattern = r"Elementos de listas:(.*?)- Vanessa" # Matchea todo desde "Elementos de listas:" hasta "- Vanessa"
match = re.search(pattern, reseña, re.DOTALL) # Permite que el punto coincida con saltos de línea

if match:
    contenido = match.group(1) # Elimina los delimitadores
    texto_listas = "\n".join( # Une las líneas que empiezan con "- " y elimina el guion y el espacio
        line[2:].strip() for line in contenido.splitlines() if line.strip().startswith("- ")
    )

Ahora que ya tenemos todo el texto útil en distintas variables de texto vamos a unificarlas y utilizar herramientas de fragmentado 

In [7]:
#unimos todo el texto en una sola variable
texto_reseña = texto_parrafos + texto_comentarios + "\n " + texto_listas

Para esta tarea de segmentación vamos a utilizar Stanza.

In [ ]:
import stanza

# Descargamos el modelo español
stanza.download('es') 
# Inicializamos el pipeline de procesamiento español
nlp = stanza.Pipeline('es')

/home/pedro/Repositories/NLP-TP1-P2/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-15 14:06:46 INFO: Downloaded file to /home/pedro/stanza_resources/resources.json
2025-05-15 14:06:46 INFO: Downloading default packages for language: es (Spanish) ...
2025-05-15 14:06:49 INFO: File exists: /home/pedro/stanza_resources/es/default.zip
2025-05-15 14:06:57 INFO: Finished downloading models and saved to /home/pedro/stanza_resources
2025-05-15 14:06:57 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-05-15 14:06:57 INFO: Downloaded file to /home/pedro/stanza_resources/resources.json
2025-05-15 14:06:59 INFO: Loading these models for language: es (Sp

In [11]:
# Analizamos el texto
document = nlp(texto_reseña)

# Mostramos cada oración del documento
print([sentence.text for sentence in document.sentences])

['Como experto artesano, tienes en mente realizar una obra de arte con tu próxima vidriera, que será colocada en la Sagrada Familia.', 'Los jugadores alternarán turnos escogiendo piezas de cristal, representadas mediante dados.', 'Tendrán que escoger cuidadosamente, ya que las vidrieras tienen colores y opacidades únicas, de forma que un dado nunca podrá colocarse adyacente a otro que comparta alguna de estas características.', 'Dispondrás de herramientas que te ayudarán a lograr completar esta importante tarea.', 'Aumenta tu prestigio adaptando la obra a los deseos de los admiradores, pero siempre manteniendo un estilo propio mientras completas tu próxima obra maestra del vidrio.', 'Así se nos presenta este Sagrada, un diseño de Daryl Andrews (The Walled City: Londonderry & Bordelands o Roar: King of the Pride) y Adrian Adamescu (Rival Kings Vefore the Earth Explodes).', 'El juego fue publicado en 2017 por Floodgate Games, tras financiarse mediante una exitosa campaña de micromecenazg

In [14]:
len(document.sentences) #Número de oraciones

311

In [43]:
#Eliminamos las oraciones repetidas
oraciones = [sentence.text for sentence in document.sentences]
oraciones_unicas = list(set(oraciones))
#Mostramos las oraciones únicas
print(len(oraciones_unicas))

257


In [ ]:
#Guardamos el texto en un archivo temporal
with open("./texto_reseña.txt", 'w') as f:
    f.write(str([sentence.text for sentence in document.sentences]))

In [20]:
oraciones = [sentence.text for sentence in document.sentences] #Guardamos las oraciones en una variable

El resultado de nuestro procesamiento del texto no deja poco más de 250 oraciones únicas para utilizar, una cantidad suficiente para que podamos hacer suficientes embeddings.

In [16]:
from sentence_transformers import SentenceTransformer, util

In [84]:
# Cargamos el modelo preentrenado multilingüe
modelo_cased_v2 = SentenceTransformer('distiluse-base-multilingual-cased-v2')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [86]:
#Procesamos las oraciones
embeddings_cased_v2 = modelo_cased_v2.encode(oraciones, convert_to_tensor=True)

In [ ]:
from sentence_transformers import util

In [87]:
embedding = modelo_cased_v2.encode("cómo jugar?") #Codificamos la frase de entrada

In [88]:
#Calculamos la similitud entre la frase de entrada y las oraciones
similarity_scores = util.cos_sim(embedding, embeddings_cased_v2) #Calculamos la similitud entre la frase de entrada y las oraciones
# Convertimos los scores a una lista
similarity_scores = similarity_scores[0].tolist() #Convertimos los scores a una lista
# Ordenamos las oraciones por similitud
oraciones_ordenadas = sorted(zip(oraciones_unicas, similarity_scores), key=lambda x: x[1], reverse=True) #Ordenamos las oraciones por similitud
# Mostramos las 5 oraciones más similares
for oracion, score in oraciones_ordenadas[:5]:
    print(f"Oración: {oracion}\nSimilitud: {score:.4f}\n")

Oración: Si el empate persiste, el vencedor será el jugador más alejado del jugador inicial (en el sentido de las agujas del reloj) en la última ronda.
Similitud: 0.4527

Oración: Una vez que todos los jugadores han dispuesto de un turno, se procede de igual forma, pero ahora comenzando por el jugador sentado a la derecha del jugador inicial y continuando en el sentido contrario a las agujas del reloj.
Similitud: 0.2968

Oración: Las sensaciones son similares.
Similitud: 0.2701

Oración: Cada jugador dispondrá de un objetivo secreto, aparte de los tres comunes que se colocan en el centro de la mesa.
Similitud: 0.2701

Oración: Gana los puntos correspondientes a los objetivos públicos.
Similitud: 0.2691

